In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
#pip install keras-bert

In [ ]:
#!pip install --upgrade tensorflow

In [2]:
tf.version.VERSION

'2.3.0'

In [3]:
import os
import warnings
import sys
import codecs
import numpy as np
import argparse
import json
import pickle

from util import read_passages, evaluate, make_folds, clean_words, test_f1, to_BIO, from_BIO, from_BIO_ind, arg2param

import tensorflow as tf
#config = tf.ConfigProto()
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1.0
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#sess = tf.Session(config=config)
sess = tf.compat.v1.Session()
#import keras.backend as K
from tensorflow.compat.v1.keras import backend as K
K.set_session(sess)
from keras.activations import softmax
from keras.regularizers import l2
from keras.models import Model, model_from_json
from keras.layers import Input, LSTM, Dense, Dropout, TimeDistributed, Bidirectional
from keras.callbacks import EarlyStopping,LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam, RMSprop, SGD
from crf import CRF
from attention import TensorAttention
from custom_layers import HigherOrderTimeDistributedDense
from generator import BertDiscourseGenerator
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

import discourse_tagger_generator_bert
import bert
from transformers import AutoTokenizer, AutoModel 
#!pip install transformers
from transformers import pipeline

In [7]:
# from transformers import *

# tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")  
# scibert = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

#tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
#model = AutoModel.from_pretrained('allenai/scibert_scivocab_cased')

In [ ]:
#%run discourse_tagger_generator_bert

In [ ]:
#pip install -r requirements.txt

In [8]:
# save_path = "allenai/"
# if not os.path.exists(save_path):
#     os.makedirs(save_path)
# tokenizer.save_pretrained(save_path)
# scibert.save_pretrained(save_path)

In [17]:
#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train_subset.txt --validation_file val_subset.txt --save --maxseqlen 40 --maxclauselen 60 
#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train_small.txt --validation_file val_small.txt --att_context LSTM_clause --save --maxseqlen 40 --maxclauselen 60 #Works
#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train.txt --validation_file val_subset.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20 #run 2/22 works
%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train.txt --validation_file val_subset.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20

#Works
#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train.txt --validation_file val_subset.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 #Works

#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train_subset.txt --validation_file val_subset.txt --save --maxseqlen 40 --maxclauselen 60 #Works
#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train.txt --validation_file train_subset.txt --save --maxseqlen 40 --maxclauselen 60 #This Works
#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train.txt --validation_file train_subset.txt --test_file test.txt --use_attention --att_context LSTM_clause --bidirectional --crf --save --maxseqlen 40 --maxclauselen 60
#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train.txt --validation_file train_subset.txt --test_file test.txt --use_attention --att_context LSTM_clause --bidirectional --crf --save --maxseqlen 40 --maxclauselen 60
#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --train_file train.txt --validation_file train_subset.txt --test_file test.txt --use_attention --crf --save --maxseqlen 0 --maxclauselen 60

#%tb

att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Using BERT.
Filtering data
Filtering data


KeyboardInterrupt: 

In [21]:
#python -u discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file text.txt --use_attention --att_context LSTM_clause --bidirectional --crf --maxseqlen 40 --maxclauselen 60
#%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file test_noDST.txt --maxseqlen 40 --maxclauselen 60
#%tb

%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file test_noDST.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20

att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Loaded model:
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 40, 768)]         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 768)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 40, 300)           230700    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 40, 700)           1822800   
_________________________________________________________________
dropout_7 (Dropout)          (None, 40, 700)           0         
_________________________________________________________________
discourse (TimeDistributed)  (None, 40, 15)            10515     
Total params: 2,064,015
Trainable params: 2,064,0

In [23]:
#test an actual research highlight text:

%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file open_cc_highlights.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20

att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Loaded model:
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 40, 768)]         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 768)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 40, 300)           230700    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 40, 700)           1822800   
_________________________________________________________________
dropout_7 (Dropout)          (None, 40, 700)           0         
_________________________________________________________________
discourse (TimeDistributed)  (None, 40, 15)            10515     
Total params: 2,064,015
Trainable params: 2,064,0

In [1]:
%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file genomicEvolutionHighLights.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20


att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. 

In [1]:
%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file genomicEvolutionRaw_test.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20


att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. 

C:\Projects\ScientificDiscourseTagging\discourse_tagger_generator_bert.py:122: UserWarning: Test sequence too long. Ignoring 85 clauses at the beginning and labeling them none.
  warnings.warn("Test sequence too long. Ignoring %d clauses at the beginning and labeling them none." % num_ignored_clauses)


In [4]:
%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file open_cc_NEUR_highlights.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20

att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Loaded model:
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 40, 768)]         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 768)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 40, 300)           230700    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 40, 700)           1822800   
_________________________________________________________________
dropout_7 (Dropout)          (None, 40, 700)           0         
_________________________________________________________________
discourse (TimeDistributed)  (None, 40, 15)            10515     
Total params: 2,064,015
Trainable params: 2,064,0

In [5]:
%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file open_cc_MEDI_highlights.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20

att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Loaded model:
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 40, 768)]         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 768)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 40, 300)           230700    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 40, 700)           1822800   
_________________________________________________________________
dropout_7 (Dropout)          (None, 40, 700)           0         
_________________________________________________________________
discourse (TimeDistributed)  (None, 40, 15)            10515     
Total params: 2,064,015
Trainable params: 2,064,0

In [7]:
%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file open_cc_BIOC_highlights.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20

att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Loaded model:
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 40, 768)]         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 768)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 40, 300)           230700    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 40, 700)           1822800   
_________________________________________________________________
dropout_7 (Dropout)          (None, 40, 700)           0         
_________________________________________________________________
discourse (TimeDistributed)  (None, 40, 15)            10515     
Total params: 2,064,015
Trainable params: 2,064,0

In [2]:
%run discourse_tagger_generator_bert.py --repfile scibert_scivocab_uncased --test_file frailRenin-AngiotensinSystem.txt --att_context LSTM_clause --bidirectional --save --maxseqlen 40 --maxclauselen 60 --batch_size 20

att=False_cont=LSTM_clause_lstm=False_bi=True_crf=False
Loaded model:
Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 40, 768)]         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 768)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 40, 300)           230700    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 40, 700)           1822800   
_________________________________________________________________
dropout_7 (Dropout)          (None, 40, 700)           0         
_________________________________________________________________
discourse (TimeDistributed)  (None, 40, 15)            10515     
Total params: 2,064,015
Trainable params: 2,064,0